# So what?

In [ ]:
import sympy as sp
import numpy as np
from sympy.interactive import printing
printing.init_printing(use_latex = True)

In [ ]:
S_Pr_Tr = sp.Symbol("S(Pr, Tr)")
G_Pr_T  = sp.Symbol("G(Pr, T)")
G0,T,Tr = sp.symbols('G_0, T, Tr')
c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8 = sp.symbols('c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8')
b_1, b_5, b_6, b_7, b_8                = sp.symbols('b_1, b_5, b_6, b_7, b_8')

## Heat capacity at a given temperature and 1 bar

In Holland and Powell datasets, the heat capacity is expressed as the following polinomial with only $c_1$ to $c_5$ terms

$$ C_p = c_1 +c_2T +\frac{c_3}{T^2} + c_4T^2 + \frac{c_5}{\sqrt{T}} + \frac{c_6}{T} + \frac{c_7}{T^3} + c_8T^3 $$



Let's evaluate this for the case of forsterite in hp02ver.dat. It compares well (to the forth decima) with the value retrieved from frendly (175.0651)

In [ ]:
def Cp(T):
    return c_1 +c_2*T +c_3/T**2 + c_4*T**2 + c_5/sp.sqrt(T) + c_6/T + c_7/T**3 + c_8*T**3
Cp(1000).subs({c_1:233.3, c_2:.1494E-2, c_3:-603800, c_4:0, c_5:-1869.7 , c_6:0, c_7:0, c_8:0}).evalf().round(4)

In [ ]:
Cp(T)

## Entropy at a given temperature and 1 bar
We can now integrate the symbolic expression to compute the intregal $C_P/T$

In [ ]:
CpoT_int = sp.integrate(Cp(T)/T,(T,T_r,T))

In [ ]:
CpoT_int

In [ ]:
fo = {'G0':-2053138, 'S0': 95.1, 'V0': 4.366}

We lambdify the expression to make it faster

In [ ]:
T_r = 298.15
int_CpoT = sp.lambdify(T, CpoT_int.subs({c_1:233.3, c_2:.1494E-2, c_3:-603800, 
                                       c_4:0, c_5:-1869.7 , c_6:0, c_7:0, c_8:0}))

and then finally we compute the entropy at 1 bar by adding the reference entropy. That compares with the frendly value (277.0713 J/mol/K)

In [ ]:
fo_S = fo.get('S0') + int_CpoT(1000)
fo_S

## Apparent gibbs enery at a given temperature and 1 bar

We can do similarly with enthalpy

In [ ]:
Cp_int   = sp.integrate(Cp(T),(T,T_r,T))

In [ ]:
Cp_int

In [ ]:
int_Cp   = sp.lambdify(T, Cp_int.subs({c_1:233.3, c_2:.1494E-2, c_3:-603800, 
                                       c_4:0, c_5:-1869.7 , c_6:0, c_7:0, c_8:0}))

In [ ]:
def fo_G(T): 
    return(fo.get('G0') + fo.get('S0')*(T_r-T) - T*int_CpoT(T) + int_Cp(T))

that compares also very well with the value from frendly (-2192536 J/mol)

In [ ]:
fo_G(1000).round()

## Apparent gibbs enery at a given temperature and pressure
### Volume at high pressure using the Murnaghan equation
It suffices to add to the above expression the volume integral
Holland and Powell expression
$$ \alpha_T = a^0 \left( 1 - \frac{10}{\sqrt(T)} \right) $$
$$ v(T,1) = v^0 [1 + a^0(T-T_r)-20a^0(\sqrt T-\sqrt T_r)] $$

in Perplex these are written as
$$ \alpha (T,P_r) = b_1 + b_5/\sqrt T $$

In [ ]:
alpha, beta, kappa, kappa1, P, v, t, v_0 = sp.symbols("alpha, beta, \kappa, \kappa', P, v, t, v_0",
                                                     positive=True, real=True)

In [ ]:
def alpha_T(T):
    return b_1 + b_5/sp.sqrt(T)

def alpha_T_int(t):
    return sp.integrate(alpha_T(t),(t,T_r,T))

def v_1(T): 
    return v_0.subs({v_0:4.366})*(1 + sp.integrate(alpha_T(t),(t,T_r,T)).subs({t:1000, b_1:.613E-4, b_5:-.613E-3})).evalf()

$$ V(T,P_r) = V_0·\left [1 + \int_{T_r}^{T}\alpha(T,P_r)dT \right] $$

In [ ]:
v_1(1000).round(6)

that compares with the frendly value 4.476770 J/mol/bar

compresibility (1/bar) $\equiv v_0/\beta$
expansivity (1/K) $\equiv v_0*\alpha_T$

In [ ]:
expansivity = 4.366*alpha_T(1000).subs({v_0:4.366, b_1:.613E-4, b_5:-.613E-3, b_6:1250000, b_7:-187.5}).evalf()
expansivity

that compares with the frendly value (0.1829078E-03 1/K) at 1 bar 1000 K

$$K(T,P_r) = b_6 + b_7·(T-T_r) $$

Molar volume at T and P
$$ v(T,P) = v(T,1)\left [ 1-\frac{4P}{\kappa_T + 4P} \right]^{1/4} $$

In [ ]:
def kappa(T):
    return b_6 + b_7*(T-T_r)

In [ ]:
def v(T,P):
    return v_1(T)*(1-4*P/(kappa(T)+ 4*P))**(1/4)

In [ ]:
v(1000,10000).subs({b_1:.613E-4, b_5:-.613E-3, b_6:1250000, b_7:-187.5}).round(6)

that compares well with the value of frendly 4.437839 J/mol/bar

### Contribution of the volume at P and T to the gibbs energy 

$$ \int_1^PV(T,P)dP= \frac{V(T,1)K_T}{3}\left [\left (1+\frac{4P}{K_T} \right)^{3/4}-1 \right]$$

In [ ]:
def int_v(T,P):
    return v_1(T)*kappa(T)/3*((1+4*P/kappa(T))**(3/4)-1)

In [ ]:
int_v(T,P)

In [ ]:
def fo_G(T, P): 
    return fo.get('G0') + fo.get('S0')*(T_r-T) - T*int_CpoT(T) + int_Cp(T) + int_v(T,P)

In [ ]:
fo_G(1000,10000).subs({v_0:4.331754, b_6:1250000, b_7:-109.5}).round()

that comapares quite well with the value from frendly -2147968 J

# All together now

Below a more condensed expression to compute the gibbs energy of an end-member, though not very elegant. 

In [ ]:
import sympy as sp
import numpy as np
from sympy.interactive import printing
printing.init_printing(use_latex = True)

T_r = 298.15

S_Pr_Tr = sp.Symbol("S(Pr, Tr)")
G0,T,Tr = sp.symbols('G_0, T, Tr')

c_1, c_2, c_3, c_5 = sp.symbols('c_1, c_2, c_3, c_5')
b_1, b_5, b_6, b_7 = sp.symbols('b_1, b_5, b_6, b_7')
alpha, beta, kappa, P, v_0  = sp.symbols("alpha, beta, \kappa, P, v_0")

def Cp(T):
    return c_1 +c_2*T +c_3/T**2 + c_5/sp.sqrt(T)

In [ ]:
def G(t,p,phase):

    CpoT_int = sp.integrate(Cp(T)/T,(T,T_r,T)).subs({c_1:phase.get('c_1'), 
                                                 c_2:phase.get('c_2'), 
                                                 c_3:phase.get('c_3'), 
                                                 c_5:phase.get('c_5')})

    Cp_int   = sp.integrate(Cp(T),(T,T_r,T)).subs({c_1:phase.get('c_1'), 
                                               c_2:phase.get('c_2'), 
                                               c_3:phase.get('c_3'), 
                                               c_5:phase.get('c_5')})


    alpha_T_int = (sp.integrate(phase.get('b_1') 
                                + phase.get('b_5')/sp.sqrt(T),(T,T_r,T)))
    
    v_1         = phase.get('V0')*(1 + alpha_T_int)
    kappa       = phase.get('b_6') + phase.get('b_7')*(T-T_r)

    int_v       = v_1*kappa/3*((1+4*P/kappa)**(3/4)-1)
    return (phase.get('G0') + phase.get('S0')*(T_r-t) - t*CpoT_int.subs({T:t}) + Cp_int.subs({T:t}) 
            + int_v.subs({T:t,P:p}))

In [ ]:
fo = {'G0' :-2053138, 
      'S0' : 95.1, 
      'V0' : 4.366, 
      'c_1': 233.3, 'c_2':.1494E-2, 'c_3':-603800, 'c_5':-1869.7, 
      'b_1':.613E-4,'b_5':-.613E-3, 'b_6':1250000, 'b_7':-109.5
     }

In [ ]:
G(1000,1000,fo).evalf()